In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import json
from os.path import exists
from keras.applications import EfficientNetV2B1

from numpy.random import choice as choose

import sys
sys.path.insert(0, "../Src/")
import basicUtil
import transferUtil

%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


Autosaving every 5 seconds


In [2]:
imageShape = (180,180)
inputShape = [imageShape[0],imageShape[1],3]
train = transferUtil.loadDataWithResizeShape("training",imageShape)
dev,test = transferUtil.loadDataWithResizeShape("testing",imageShape,.5)

base_model = EfficientNetV2B1(
    include_top=False,
    classes=len(train.class_names),
    classifier_activation=None,
    include_preprocessing=False,
    input_shape=inputShape
)
for layer in base_model.layers:
    layer.trainable = False

possibleNeuronsPerLayer = basicUtil.createRangeFromMidpoint(16,32)
possibleEpochs = basicUtil.createRangeFromMidpoint(5,10)

dropoutCriticalPoints = (0,1)
adamLearningRateCriticalPoints = (1e-4,1e-2)
L2CriticalPoints = (1e-2,1e3) 

n_NeuronsPerFCLayers = []
n_Epochs = []
dropoutRates = []
adamLearningRates = []
L2Rates = []
trainScores = []
devScores = []

trial = 0
bestDevScore = 0
oversizedNeuronNumbers = []
oversizedEpochNumbers = []

while trial < 100:
    numberOfNeuronsPerFCLayer = choose(possibleNeuronsPerLayer)
    numberOfEpochs = choose(possibleEpochs)
    
    dropoutRate = basicUtil.generateDropoutRate(dropoutCriticalPoints[0],dropoutCriticalPoints[1])
    adamLearningRate = basicUtil.generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
    L2Rate = basicUtil.generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
    model = transferUtil.createTransferableModel(base_model, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate, L2Rate)
    
    model.fit(train,epochs=numberOfEpochs,verbose=0)

    model_path = f'../Models/EffecientNetTrials/effecientnet_model_{trial}.h5'
    model.save(model_path)
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    if model_size < 40:
        oversizedNeuronNumbers = []
        oversizedEpochNumbers = []
        print('trainScore')
        trainScore = model.evaluate(train)[1]
        print('devScore')
        devScore = model.evaluate(dev)[1]

        if (devScore > 0.91) and (devScore > bestDevScore):
            testScore = model.evaluate(test)[1]
            model_path = f'../Models/best_effecientnet_model_.h5'
            model.save(model_path)
            bestModelParams = {
                'n_NeuronsPerFCLayers' : int(numberOfNeuronsPerFCLayer),
                'n_Epochs' : int(numberOfEpochs),
                'dropoutRate' : dropoutRate,
                'adamLearningRates' : adamLearningRate,
                'L2Rates' : L2Rate,
                'modelSize' : model_size,
                'trainScore': trainScore,
                'devScore': devScore,
                'testScore': testScore
            }
            with open('../Models/best_effecientnet_model_params.json', 'w') as f:
                json.dump(bestModelParams, f)
            bestDevScore = devScore       

        n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
        n_Epochs.append(numberOfEpochs)
        
        adamLearningRates.append(adamLearningRate)
        dropoutRates.append(dropoutRate)
        L2Rates.append(L2Rate)
        trainScores.append(trainScore)
        devScores.append(devScore)
        
        print('concluding trial ',trial)
        trial += 1
    else:
        print(f'redoing trial {trial}. Model was {model_size}MB.')
        failedTrial = transferUtil.createTransferModelParametersDF([numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                              [dropoutRate],[adamLearningRate],[L2Rate],[np.nan],[np.nan])
        display(failedTrial)
        oversizedNeuronNumbers.append(numberOfNeuronsPerFCLayer)
        oversizedEpochNumbers.append(numberOfEpochs)
        if len(oversizedNeuronNumbers) >= 3:
            possibleNeuronsPerLayer = [x for x in possibleNeuronsPerLayer if x < np.mean(oversizedNeuronNumbers)]
            possibleEpochs = [x for x in possibleEpochs if x < np.mean(oversizedEpochNumbers)]
            oversizedNeuronNumbers = []
            oversizedEpochNumbers = []
        
    if (trial % 10 == 9): 
        modelParametersDF = transferUtil.createTransferModelParametersDF(n_NeuronsPerFCLayers,n_Epochs,
                                                dropoutRates,adamLearningRates,L2Rates,trainScores,devScores)
        modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
        display(modelParametersDF)
        
        top5 = modelParametersDF[0:5]
        possibleNumberOfNeuronsPerFCLayer = basicUtil.getAdjustedRange(top5['n_NeuronsPerFCLayers'])
    
        possibleNumberOfEpochs = basicUtil.getAdjustedRange(top5['n_Epochs'])
        dropoutCriticalPoints = basicUtil.calculateCriticalPoints(top5['dropoutRate'])
        adamLearningRateCriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['adamLearningRates'])
        L2CriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['L2Rates'])

        n_NeuronsPerFCLayers = []
        n_Epochs = []
        dropoutRates = []
        adamLearningRates = []
        L2Rates = []
        trainScores = []
        devScores = []

        if bestDevScore > 0.91:
            trial = 101
            
basicUtil.displayFinalResults('../Models/best_effecientnet_model_params.json')
    

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-08-18 18:44:18.835218: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-08-18 18:44:18.835364: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-08-18 18:44:21.125057: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


trainScore
75/75 [==============================] - 20s 254ms/step - loss: 96.3272 - f1_score: 0.6340
devScore


2024-08-18 18:46:17.758575: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-08-18 18:46:17.758717: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 3s 241ms/step - loss: 96.3155 - f1_score: 0.6461
concluding trial  0
trainScore
75/75 [==============================] - 20s 256ms/step - loss: 41.6363 - f1_score: 0.3645
devScore
10/10 [==============================] - 3s 228ms/step - loss: 41.6411 - f1_score: 0.3570
concluding trial  1
trainScore
75/75 [==============================] - 20s 256ms/step - loss: 25.8305 - f1_score: 0.8951
devScore
10/10 [==============================] - 3s 226ms/step - loss: 25.8260 - f1_score: 0.8915
concluding trial  2
trainScore
75/75 [==============================] - 21s 259ms/step - loss: 2.2665 - f1_score: 0.9720
devScore
10/10 [==============================] - 3s 228ms/step - loss: 2.2655 - f1_score: 0.9720


...

,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
0,28,3,0.04921,0.005641,9.879502,NaN,NaN


trainScore
75/75 [==============================] - 21s 264ms/step - loss: 1.1887 - f1_score: 0.5117
devScore
10/10 [==============================] - 3s 230ms/step - loss: 1.1819 - f1_score: 0.5127
concluding trial  6
trainScore
75/75 [==============================] - 21s 262ms/step - loss: 123.2569 - f1_score: 0.3850
devScore
10/10 [==============================] - 3s 235ms/step - loss: 123.2219 - f1_score: 0.3985
concluding trial  7
trainScore
75/75 [==============================] - 21s 268ms/step - loss: 0.8790 - f1_score: 0.9714
devScore
10/10 [==============================] - 3s 241ms/step - loss: 0.8780 - f1_score: 0.9714
concluding trial  8


,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
4,14,4,0.397426,0.000117,0.575081,0.997517,0.992983
3,6,3,0.806847,0.006425,0.146666,0.972031,0.972031
8,1,4,0.447016,0.004339,0.037006,0.971413,0.971414
2,8,6,0.118347,0.001572,318.624933,0.895106,0.891512
5,14,1,0.746521,0.000115,101.000317,0.741121,0.761996
0,20,5,0.714696,0.004129,2.322628,0.634020,0.646111
6,9,6,0.086905,0.000240,50.485991,0.511653,0.512741
7,20,2,0.429993,0.002040,124.406424,0.384978,0.398495
1,22,4,0.853411,0.000751,291.755757,0.364535,0.356966


Final Model


,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,modelSize,trainScore,devScore,testScore
0,14,4,0.397426,0.000117,0.575081,34.361641,0.997517,0.992983,0.997517
